In [1]:
# Apagar a pasta caso existir
import shutil
shutil.rmtree('projeto-final', ignore_errors=True)
shutil.rmtree('sample_data', ignore_errors=True)

# Criar pasta e clonar o projeto
!mkdir projeto-final
!git clone https://github.com/franciscosens/bcc-processamento-imagens-trabalho-final projeto-final --quiet

# Copiar arquivos necessários
!cp projeto-final/requirements.txt requirements.txt
!cp projeto-final/setup.py setup.py
!cp projeto-final/setup.cfg setup.cfg
!cp projeto-final/domino.py domino.py
!cp -r projeto-final/mrcnn mrcnn

# Apagar projeto clonado
shutil.rmtree('projeto-final', ignore_errors=True)

In [ ]:
%tensorflow_version 1.x
%matplotlib inline 

In [ ]:
!pip install -r requirements.txt
!python setup.py install

In [18]:
# Apagar diretórios
import shutil
shutil.rmtree('dataset', ignore_errors=True)

# Criação dos diretórios necessários
!mkdir dataset
!mkdir dataset/train dataset/val

# Criação da pasta para armazenar os pesos da rede neural
!mkdir logs

In [19]:
# Download das imagens de treinamento e de teste
!wget https://raw.githubusercontent.com/franciscosens/bcc-processamento-imagens-trabalho-final/master/dataset/val.zip -c --quiet
!wget https://raw.githubusercontent.com/franciscosens/bcc-processamento-imagens-trabalho-final/master/dataset/test.zip -c --quiet

# Descompactar imagens
!unzip -qq val.zip -d dataset/val/
!unzip -qq test.zip -d test/

!rm val.zip
! rm test.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!gsutil cp "/content/drive/My Drive/Processamento de Imagens Projeto Final/mask_rcnn_shapes3.h5" logs/

In [16]:
from mrcnn.config import Config

class DominoConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "dom"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + domino

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
from domino import DominoDataset
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import json


config = DominoConfig()

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

custom_WEIGHTS_PATH = "logs/mask_rcnn_shapes3.h5"

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


custom_DIR = os.path.join(ROOT_DIR, "dataset")

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

# Load validation dataset
dataset = DominoDataset()
dataset.load_domino(custom_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)

In [21]:
from mrcnn import model as modellib, utils

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs/")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_shapes3.h5")



DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
# Load a random image from the images folder
try:
    from cv2 import cv2 as cv2
except ImportError:
    pass

from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)
folder_name = "test/test2"

IMAGE_DIR = os.path.join(ROOT_DIR, folder_name)
data_path = os.path.join(IMAGE_DIR ,'*g')
file_names =  glob.glob(data_path)

quantidade_total = len(file_names)
quantidade = 0
for file_name in file_names:
  posicao = file_name.rindex('/')
  
  file_name = file_name[posicao+1:len(file_name)]
  image = skimage.io.imread(os.path.join(IMAGE_DIR, file_name))

  # Run detection
  results = model.detect([image], verbose=1)

  # Visualize results
  r = results[0]
  imagem = visualize.process_instances_and_return_image(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'])
  cv2.imwrite('/content/resultado/' + file_name, imagem)
  quantidade = quantidade + 1
  print(f'{quantidade} de {quantidade_total}')
  break

In [ ]:
!zip -r /content/resultado.zip /content/resultado
from google.colab import files
files.download("/content/resultado.zip")